### Efficientnet_b3迁移学习

In [1]:
import json
from PIL import Image
import torch
from torchvision import transforms
import wget

In [2]:

import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import datasets,transforms,models
import os
import shutil
import time
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [5]:
model = torch.hub.load('rwightman/gen-efficientnet-pytorch',
                       'efficientnet_b3', pretrained=True)

Using cache found in C:\Users\King/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b3_ra-a5e2fbc7.pth" to C:\Users\King/.cache\torch\checkpoints\efficientnet_b3_ra-a5e2fbc7.pth


In [7]:
data_transform=transforms.Compose([
    transforms.Resize(320),
    transforms.CenterCrop(320),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.488,0.455,0.417],std=[0.230,0.225,0.225])#RGB
    #transforms.Normalize(mean=[0.415,0.454,0.487],std=[0.225,0.225,0.230])#
])
#训练数据读取接口
train_dataset=datasets.ImageFolder(root='./copy/train/',
                                  transform= data_transform)
#把数据打包成batch_size大小的tensor
train_loader=torch.utils.data.DataLoader( train_dataset,
                                        batch_size=64,
                                        shuffle=True)
#验证集数据读取接口
val_dataset=datasets.ImageFolder(root='./copy/val/',
                                 transform=data_transform)
val_loader=torch.utils.data.DataLoader(val_dataset,
                                       batch_size=64,
                                       shuffle=True)

In [8]:
print(val_dataset.class_to_idx)

{'cats': 0, 'dogs': 1}


In [9]:
for param in model.parameters():
    param.requires_grad=False

In [11]:
model.classifier=nn.Sequential(nn.Linear(1280,2))

In [12]:
model=model.cuda()
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.classifier.parameters(),lr=1e-4)

In [22]:
def val():
    correct=0
    val_loss=0.0
    val_total=0
    model.eval()

    for i,data in enumerate(val_loader):
        images,labels=data
        images,labels=Variable(images.cuda()),Variable(labels.cuda())
        outputs=model(images)
        _,predicted=torch.max(outputs.data,1)
        loss=criterion(outputs,labels)
        val_loss+=loss.item()
        val_total+=labels.size(0)
        correct+=(predicted==labels.data).sum()

#     print('val %d epoch loss: %.3f  acc:%4.2f ' %(epoch+1,
#                                                  val_loss/(i+1),
#                                                  100*float(correct)/float(val_total)))
    return val_loss/(i+1),100*float(correct)/float(val_total)

In [23]:
def train():
    
    for epoch in range(epochs):
        model.train()
        running_loss=0.0
        train_correct=0
        train_total=0
        
        for i,data in enumerate(train_loader,0):#一个epoch完成后结束for循环
            inputs,train_labels=data
            inputs,labels=Variable(inputs.cuda()),Variable(train_labels.cuda())
            outputs=model(inputs)#一个batch的输出
            
            #根据网络输出计算此时的准确率
            _,train_predicted=torch.max(outputs.data,1)
            train_correct+=(train_predicted==labels.data).sum()#每个batch中正确的个数，累加器
            
            loss=criterion(outputs,labels)
            
            #计算损失函数中的L2正则化项
#             L2_reg=0
#             for param in model.parameters():
#                 L2_reg+=torch.norm(param)
            
#             loss+=0.1*L2_reg
            
            optimizer.zero_grad() 
            loss.backward()
            optimizer.step()
            running_loss+=loss.item() #损失值，累加器
            train_total+=train_labels.size(0)#batch图像个数，累加器
            
            if (i+1)%print_per_batches==0: #每隔10个batch就记录一次这10个batch的平均损失值
                val_loss,val_acc=val()
                print('[%d %5d] train loss: %.3f  train acc: %.3f  val loss: %.3f  val acc: %.3f'
                      %(epoch+1,i+1,running_loss/print_per_batches,100*float(train_correct)/float(train_total),val_loss,val_acc))
                train_loss_batch.append(running_loss/print_per_batches)
                val_loss_batch.append(val_loss)
                running_loss=0
                train_correct=0
                train_total=0
                
               
        
        #print('train %d epoch  acc: %4.2f'%(epoch+1, 100*float(train_correct)/float(train_total)))
        
        
        
        #在训练完一个epoch后，用验证集进行验证
        
        
        if epoch % save_every==0:
            torch.save(model.state_dict(),'./models/model-%d.pkl' % epoch)



In [18]:
start=time.time()
val_loss,val_acc=val()
print(time.time()-start)

42.234071493148804


In [24]:
train_loss_batch=[]# 记录训练过程中的损失值
val_loss_batch=[]
save_every=1 #每一个epoch保存一下网络参数值
epochs=5 
print_per_batches=20

# 开始训练，并记录训练消耗的时间
time_open=time.time()
train()
time_end=time.time()
time_used=time_end-time_open
print('time used : {} minutes'.format(time_used/60))

[1    20] train loss: 0.296  train acc: 93.906  val loss: 0.286  val acc: 94.360
[1    40] train loss: 0.281  train acc: 94.219  val loss: 0.274  val acc: 94.600
[1    60] train loss: 0.262  train acc: 95.000  val loss: 0.262  val acc: 94.680
[1    80] train loss: 0.261  train acc: 94.609  val loss: 0.255  val acc: 94.720
[1   100] train loss: 0.259  train acc: 93.438  val loss: 0.246  val acc: 94.920
[1   120] train loss: 0.245  train acc: 95.234  val loss: 0.243  val acc: 94.920
[1   140] train loss: 0.238  train acc: 93.984  val loss: 0.236  val acc: 94.880
[1   160] train loss: 0.231  train acc: 94.062  val loss: 0.225  val acc: 94.880
[1   180] train loss: 0.221  train acc: 95.625  val loss: 0.222  val acc: 95.200
[1   200] train loss: 0.223  train acc: 95.078  val loss: 0.219  val acc: 95.000
[1   220] train loss: 0.204  train acc: 95.781  val loss: 0.209  val acc: 95.200
[1   240] train loss: 0.200  train acc: 96.562  val loss: 0.207  val acc: 95.160
[1   260] train loss: 0.203 

In [ ]:
x=range(1,len(train_loss_batch)+1)
plt.plot(x,train_loss_batch,label='train loss')
plt.plot(x,val_loss_batch,label='val loss')
plt.legend(loc='upper right')
plt.show()

In [66]:
f=open('efficient_trainLoss.txt','w')
f.write(str(train_loss_batch))
f.close()

In [67]:
f=open('efficient_valLoss.txt','w')
f.write(str(val_loss_batch))
f.close()